In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import torch
import kagglehub
import os

In [ ]:
path = kagglehub.dataset_download("crowdflower/twitter-airline-sentiment")
print("Path to dataset files:", path)

csv_path = os.path.join(path, "Tweets.csv")
df = pd.read_csv(csv_path)
df = df[['text', 'airline_sentiment']]
df = df.rename(columns={'airline_sentiment': 'sentiment'})
df = df.dropna()
df = df.sample(1000, random_state=42)
df.head()

In [ ]:
def preprocess(text):
    text = text.lower()
    text = text.replace('\n', ' ')
    return text

df['text_clean'] = df['text'].apply(preprocess)

In [ ]:
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
sentiment_pipe = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [ ]:
results = sentiment_pipe(list(df['text_clean'][:20]))
for text, res in zip(df['text_clean'][:20], results):
    print(f"Testo: {text}\nSentiment: {res}\n")

In [ ]:
df['predicted_sentiment'] = [r['label'] for r in sentiment_pipe(list(df['text_clean']))]
df.head()

In [ ]:
label_map = {
    'positive': 'positive',
    'neutral': 'neutral',
    'negative': 'negative',
    'LABEL_0': 'negative',
    'LABEL_1': 'neutral',
    'LABEL_2': 'positive'
}
df['predicted_sentiment'] = df['predicted_sentiment'].map(label_map)
df['sentiment'] = df['sentiment'].map(label_map)

In [ ]:
print(classification_report(df['sentiment'], df['predicted_sentiment']))
cm = confusion_matrix(df['sentiment'], df['predicted_sentiment'], labels=['positive', 'neutral', 'negative'])
sns.heatmap(cm, annot=True, fmt='d', xticklabels=['positive', 'neutral', 'negative'], yticklabels=['positive', 'neutral', 'negative'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
df['timestamp'] = pd.date_range(start='2023-01-01', periods=len(df), freq='H')
df['sentiment_score'] = df['predicted_sentiment'].map({'positive': 1, 'neutral': 0, 'negative': -1})

df.set_index('timestamp', inplace=True)
df['sentiment_score'].rolling(window=24).mean().plot(figsize=(12,4))
plt.title('Andamento del sentiment (media mobile 24h)')
plt.show()

In [ ]:
df['timestamp'] = pd.date_range(start='2023-01-01', periods=len(df), freq='H')
df['sentiment_score'] = df['predicted_sentiment'].map({'positive': 1, 'neutral': 0, 'negative': -1})

df.set_index('timestamp', inplace=True)
df['sentiment_score'].rolling(window=24).mean().plot(figsize=(12,4))
plt.title('Andamento del sentiment (media mobile 24h)')
plt.show()

In [ ]:
accuracy = (df['sentiment'] == df['predicted_sentiment']).mean()
if accuracy < 0.8:
    print("Trigger retraining: accuracy bassa, avviare retraining automatico!")
else:
    print("Accuracy soddisfacente, nessun retraining necessario.")

In [ ]:
# Pipeline CI/CD (Schema)
#- **Step 1:** Push su GitHub attiva workflow GitHub Actions
#- **Step 2:** Esegui test unitari e di integrazione (es. pytest)
#- **Step 3:** Esegui training/retraining del modello (script Python)
#- **Step 4:** Deploy automatico su HuggingFace o altro servizio cloud
#- **Step 5:** Monitoraggio automatico delle performance (alert se accuracy scende)

In [ ]:
# Conclusioni
#- Il sistema automatizza l’analisi del sentiment e il monitoraggio della reputazione aziendale.
#- L’integrazione MLOps consente retraining e deploy automatici, garantendo adattabilità e scalabilità.
#- Il monitoraggio continuo permette di reagire rapidamente a cambiamenti nel sentiment degli utenti.

# Documentazione
#- Tutto il codice è documentato e pronto per essere integrato in una pipeline CI/CD.
#- Per deploy su HuggingFace, consulta la [guida ufficiale](https://huggingface.co/docs/hub/spaces-sdks-docker).